# 모델 매개변수 최적화하기

* 모델은 학습의 각 반복 단계에서 결과를 추측하고, 추측과 정답 사이의 오류(손실, loss)를 계산하고, 매개변수에 대한 오류의 도함수를 수집한 뒤, 경사하강법을 이용해 파라미터들을 최적화한다.


## 기본 코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

100%|██████████| 26421880/26421880 [00:03<00:00, 8187183.53it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 134820.06it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2543445.67it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 14738755.63it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [4]:
train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

In [5]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [6]:
model = NeuralNetwork()

# Hyperparameter
* 하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수이다. 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있다.

* 학습 시에는 다음과 같은 하이퍼파라미터를 정의한다.
  * **에폭(epoch) 수**: 데이터셋을 반복하는 횟수
  * **배치 크기 (batch size)**: 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
  * **학습률(learning rate)** : 각 배치에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [7]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 최적화 단계 (Optimization Loop)

* 하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. 최적화 단계의 각 반복(iteration)을 **에폭**이라고 부른다.

* 하나의 에폭은 다음 두 부분으로 구성된다.
  1. **학습 단계 (train loop)**: 학습 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴
  2. **검증/테스트 단계 (validation/test loop)**: 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)


## 1. 학습 단계 (Trainining Loop)
### 손실 함수 (Loss function)

* **손실함수(loss function)**은 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 모델은 학습 중에 이 값을 최소화하려고한다. 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교해 손실(loss)를 계산

* 일반적인 손실함수에는 regression task에 사용하는 `nn.MSELoss` (평균 제곱 오차)나 classification에 사용하는 `nn.NLLLoss` (음의 로그 우도), 그리고 `nn.LogSoftmax`와 `nn.NLLLoss`를 합친 `nn.CrossEntropyLoss` 등이 있다.

In [10]:
# 모델의 출력 logit을 nn.CrossEntropyLoss에 전달하여 logit을 정규화하고 예측 오류를 계산해보자.
loss_fn = nn.CrossEntropyLoss()


### 옵티마이저 (Optimizer)
* 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조절하는 과정
* 최적화 알고리즘은 이 과정을 수행하는 방식을 정함 (SGD, GD, ADAM, RMSprop 등 다양한 알고리즘이 있음)
* 모든 최적화 절차(Logic)은 `optimizer` 객체에 캡슐화된다.
* 학습하려는 모델의 매개변수와 학습률(learning_rate) 하이퍼파라미터를 등록하여 옵티마이저를 초기화


In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

### Training loop에서 최적화의 단계
1. `optimizer.zero_grad()`를 호출하여 모델 매개변수의 경사도를 재설정. 기본적으로 기울기는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정한다.
2. `loss.backwards()`를 호출하여 예측 손실(prediction loss)를 역전파한다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장한다.
3. 변화도를 계산한 뒤에는 `optimizer.step()`을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정


# 전체 구현
* `train_loop`: 최적화 코드 반복 수행
* `test_loop`: 테스트 데이터로 모델의 성능 측정

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # 예측(prediction)과 손실(loss) 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad() # 모델 매개변수의 경사도 재설정
    loss.backward() #loss에 대한 역전파
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [12]:
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X,y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [13]:
# 손실 함수와 옵티마이저 초기화 & train_loop와 test_loop에 전달
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------------------------------
loss: 2.309654 [   64/60000]
loss: 2.298455 [ 6464/60000]
loss: 2.282496 [12864/60000]
loss: 2.267633 [19264/60000]
loss: 2.255124 [25664/60000]
loss: 2.239504 [32064/60000]
loss: 2.226477 [38464/60000]
loss: 2.205332 [44864/60000]
loss: 2.200482 [51264/60000]
loss: 2.162131 [57664/60000]
Test error: 
 Accuracy: 55.1%, Avg loss: 2.164208

Epoch 2
------------------------------
loss: 2.176239 [   64/60000]
loss: 2.168624 [ 6464/60000]
loss: 2.110530 [12864/60000]
loss: 2.123273 [19264/60000]
loss: 2.079950 [25664/60000]
loss: 2.030264 [32064/60000]
loss: 2.045760 [38464/60000]
loss: 1.973598 [44864/60000]
loss: 1.980638 [51264/60000]
loss: 1.908379 [57664/60000]
Test error: 
 Accuracy: 60.5%, Avg loss: 1.905910

Epoch 3
------------------------------
loss: 1.940331 [   64/60000]
loss: 1.913430 [ 6464/60000]
loss: 1.790803 [12864/60000]
loss: 1.829879 [19264/60000]
loss: 1.732815 [25664/60000]
loss: 1.680261 [32064/60000]
loss: 1.697968 [38464/60000